<a href="https://colab.research.google.com/github/kennyxu256/investigatingCryptoPumpAndDumps/blob/main/march2023DataExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Senior Year/busi 496/dataAnalysis

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive
/content/gdrive/MyDrive/Senior Year/IRP/IRPAnalysis


In [ ]:
!pip install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.1 MB/s eta 0:00:00


In [ ]:
import ccxt
import pandas as pd
import time
import os

In [ ]:
def create_ohlcv_df(data):
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(data, columns=header)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms', origin='unix')  # convert timestamp to datetime
    return df

In [ ]:
# Pulls the raw data from exchanges. Returns list of fetched symbols.
def pull_data(exchange, from_date, n_candles, c_size, f_path, skip=False):
    count = 1
    millisec = 1000
    hold = 5

    missing_symbols = []

    # -- create a folder --
    newpath = f_path + '/' + exchange + '/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    # -- load exchange --
    exc_instance = getattr(ccxt, exchange)()
    exc_instance.load_markets()
    from_timestamp = exc_instance.parse8601(from_date)

    # -- pull ohlcv --
    for symbol in exc_instance.symbols:
        for attempt in range(3):  # 3 attempts max
            try:
                print('Pulling:', exchange, ':', symbol, '[{}/{}]'.format(count, len(exc_instance.symbols)))
                data = exc_instance.fetch_ohlcv(symbol, c_size, from_timestamp, n_candles)

                # if < n_candles returned, skip this pair
                if len(data) < n_candles and skip is True:
                    continue

                # -- create DF --
                df = create_ohlcv_df(data)

                # -- save CSV --
                symbol = symbol.replace("/", "-")
                filename = newpath + '{}_{}_[{}]-TO-[{}].csv'.format(exchange, symbol, df['Timestamp'].iloc[0],
                                                                     df['Timestamp'].iloc[-1])
                filename = filename.replace(":", ".")
                df.to_csv(filename)

            except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout,
                    IndexError) as error:
                print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
                time.sleep(hold)
            else:  # if no error, proceed to next iteration
                break
        else:  # we failed all attempts
            print('All attempts failed, skipping:', symbol)
            missing_symbols.append(symbol)
            continue

        count += 1
        # -- wait for rate limit --
        time.sleep((exc_instance.rateLimit / millisec) + 5)  # rate limit +5 seconds to just to be safe

    # print out any symbols we could not obtain
    if len(missing_symbols) > 0:
        print('Unable to obtain:', missing_symbols)

    return missing_symbols

In [ ]:
from_date = '2023-03-10 00:00:00'
localFolder = 'newData'

In [ ]:
pull_data('bittrex', from_date, 480, '1h', localFolder)

NameError: ignored

In [ ]:
pull_data('kraken', from_date, 480, '1h', localFolder)

Pulling: kraken : 1INCH/EUR [1/645]
Pulling: kraken : 1INCH/USD [2/645]
Pulling: kraken : AAVE/BTC [3/645]
Pulling: kraken : AAVE/ETH [4/645]
Pulling: kraken : AAVE/EUR [5/645]
Pulling: kraken : AAVE/GBP [6/645]
Pulling: kraken : AAVE/USD [7/645]
Pulling: kraken : ACA/EUR [8/645]
Pulling: kraken : ACA/USD [9/645]
Pulling: kraken : ACH/EUR [10/645]
Pulling: kraken : ACH/USD [11/645]
Pulling: kraken : ADA/AUD [12/645]
Pulling: kraken : ADA/BTC [13/645]
Pulling: kraken : ADA/ETH [14/645]
Pulling: kraken : ADA/EUR [15/645]
Pulling: kraken : ADA/GBP [16/645]
Pulling: kraken : ADA/USD [17/645]
Pulling: kraken : ADA/USDT [18/645]
Pulling: kraken : ADX/EUR [19/645]
Pulling: kraken : ADX/USD [20/645]
Pulling: kraken : AGLD/EUR [21/645]
Pulling: kraken : AGLD/USD [22/645]
Pulling: kraken : AIR/EUR [23/645]
Pulling: kraken : AIR/USD [24/645]
Pulling: kraken : AKT/EUR [25/645]
Pulling: kraken : AKT/USD [26/645]
Pulling: kraken : ALCX/EUR [27/645]
Pulling: kraken : ALCX/USD [28/645]
Pulling: kraken

NetworkError: ignored

In [ ]:
pull_data('kucoin', from_date, 480, '1h', localFolder)

Pulling: kucoin : 1EARTH/USDT [1/1289]
Pulling: kucoin : 1INCH/USDT [2/1289]
Pulling: kucoin : 2CRZ/USDT [3/1289]
Pulling: kucoin : AAVE/BTC [4/1289]
Pulling: kucoin : AAVE/KCS [5/1289]
Pulling: kucoin : AAVE/USDT [6/1289]
Pulling: kucoin : AAVE3L/USDT [7/1289]
Pulling: kucoin : AAVE3S/USDT [8/1289]
Pulling: kucoin : ABBC/BTC [9/1289]
Pulling: kucoin : ABBC/USDT [10/1289]
Pulling: kucoin : ACA/BTC [11/1289]
Pulling: kucoin : ACA/USDT [12/1289]
Pulling: kucoin : ACE/USDT [13/1289]
Pulling: kucoin : ACH/USDT [14/1289]
Pulling: kucoin : ACOIN/USDT [15/1289]
Pulling: kucoin : ACQ/USDC [16/1289]
Pulling: kucoin : ACQ/USDT [17/1289]
Pulling: kucoin : ACS/USDT [18/1289]
Pulling: kucoin : ACT/USDT [19/1289]
Pulling: kucoin : ADA/BTC [20/1289]
Pulling: kucoin : ADA/KCS [21/1289]
Pulling: kucoin : ADA/USDC [22/1289]
Pulling: kucoin : ADA/USDT [23/1289]
Pulling: kucoin : ADA3L/USDT [24/1289]
Pulling: kucoin : ADA3S/USDT [25/1289]
Pulling: kucoin : ADS/BTC [26/1289]


RateLimitExceeded: ignored

In [ ]:
pull_data('lbank', from_date, 480, '1h', localFolder)

Pulling: lbank : 1INCH/USDT [1/949]
Pulling: lbank : 1INCH3L/USDT [2/949]
Pulling: lbank : 1INCH3S/USDT [3/949]
Pulling: lbank : 1INCH5L/USDT [4/949]
Pulling: lbank : 1INCH5S/USDT [5/949]
Pulling: lbank : 3ULL/USDT [6/949]
Pulling: lbank : 4JNET/USDT [7/949]
Pulling: lbank : 7PXS/USDT [8/949]
Pulling: lbank : A1A/USDT [9/949]
Pulling: lbank : AAVE/USDT [10/949]
Pulling: lbank : AAVE3L/USDT [11/949]
Pulling: lbank : AAVE3S/USDT [12/949]
Pulling: lbank : ABEY/USDT [13/949]
Pulling: lbank : ACA/USDT [14/949]
Pulling: lbank : ACS/USDT [15/949]
Pulling: lbank : ADA/USDT [16/949]
Pulling: lbank : ADA3L/USDT [17/949]
Pulling: lbank : ADA3S/USDT [18/949]
Pulling: lbank : AGIX/USDT [19/949]
Pulling: lbank : AGIX3L/USDT [20/949]
Pulling: lbank : AGIX3S/USDT [21/949]
Pulling: lbank : AGLD/USDT [22/949]
Pulling: lbank : AGLD3L/USDT [23/949]
Pulling: lbank : AGLD3S/USDT [24/949]
Pulling: lbank : ALGO/USDT [25/949]
Pulling: lbank : ALGO3L/USDT [26/949]
Pulling: lbank : ALGO3S/USDT [27/949]
Pulling: 

[]